In [1]:
from sqlalchemy import null


{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "14J-ylhuxukV"
   },
   "source": [
    "# Урок 10. Распознавание лиц и эмоций.\n",
    "## **Задание по итогам курса:**\n",
    "\n",
    "- (упрощенное/для тех, у кого нет вебкамеры)\n",
    "    1. Нужно написать приложение, которое будет получать на вход *изображение*.\n",
    "    2. В процессе определять, что перед камерой находится человек, задетектировав его лицо на кадре.\n",
    "    3. На изображении человек показывает жесты руками, а алгоритм должен считать их и классифицировать.\n",
    "\n",
    "- (более сложное)\n",
    "    1. Нужно написать приложение, которое будет считывать и выводить кадры с *веб-камеры*.\n",
    "    2. В процессе считывания определять что перед камерой находится человек, задетектировав его лицо на кадре.\n",
    "    3. Человек показывает жесты руками, а алгоритм должен считать их и классифицировать. \n",
    "____________________\n",
    "- Для распознавания жестов, вам надо будет скачать датасет https://www.kaggle.com/gti-upm/leapgestrecog, разработать модель для обучения и обучить эту модель.\n",
    "- Как работать с веб-камерой на google colab https://stackoverflow.com/questions/54389727/opening-web-camera-in-google-colab\n",
    "\n",
    "\n",
    "У кого нет возможности работать через каггл (нет верификации), то можете данные взять по ссылке: https://disk.yandex.ru/d/R2PGlaXDf6_HzQ"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "qdGoKxcgbCg6",
    "outputId": "2930d44e-b1a9-42a2-9d9c-49d980243f8f"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/\n",
      "Collecting opendatasets\n",
      "  Downloading opendatasets-0.1.22-py3-none-any.whl (15 kB)\n",
      "Requirement already satisfied: tqdm in /usr/local/lib/python3.8/dist-packages (from opendatasets) (4.64.1)\n",
      "Requirement already satisfied: kaggle in /usr/local/lib/python3.8/dist-packages (from opendatasets) (1.5.12)\n",
      "Requirement already satisfied: click in /usr/local/lib/python3.8/dist-packages (from opendatasets) (7.1.2)\n",
      "Requirement already satisfied: certifi in /usr/local/lib/python3.8/dist-packages (from kaggle->opendatasets) (2022.12.7)\n",
      "Requirement already satisfied: python-slugify in /usr/local/lib/python3.8/dist-packages (from kaggle->opendatasets) (8.0.0)\n",
      "Requirement already satisfied: urllib3 in /usr/local/lib/python3.8/dist-packages (from kaggle->opendatasets) (1.24.3)\n",
      "Requirement already satisfied: python-dateutil in /usr/local/lib/python3.8/dist-packages (from kaggle->opendatasets) (2.8.2)\n",
      "Requirement already satisfied: requests in /usr/local/lib/python3.8/dist-packages (from kaggle->opendatasets) (2.25.1)\n",
      "Requirement already satisfied: six>=1.10 in /usr/local/lib/python3.8/dist-packages (from kaggle->opendatasets) (1.15.0)\n",
      "Requirement already satisfied: text-unidecode>=1.3 in /usr/local/lib/python3.8/dist-packages (from python-slugify->kaggle->opendatasets) (1.3)\n",
      "Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.8/dist-packages (from requests->kaggle->opendatasets) (2.10)\n",
      "Requirement already satisfied: chardet<5,>=3.0.2 in /usr/local/lib/python3.8/dist-packages (from requests->kaggle->opendatasets) (4.0.0)\n",
      "Installing collected packages: opendatasets\n",
      "Successfully installed opendatasets-0.1.22\n"
     ]
    }
   ],
   "source": [
    "!pip install opendatasets"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,   # How to fix this error? 
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "iEV2yKw4bGV6",
    "outputId": "ce8e924c-ef40-45a6-ae58-82a13632d41a"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds\n",
      "Your Kaggle username: andreyopanasenko\n",
      "Your Kaggle Key: ··········\n",
      "Downloading leapgestrecog.zip to ./leapgestrecog\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "100%|██████████| 2.13G/2.13G [00:16<00:00, 136MB/s]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n"
     ]
    }
   ],
   "source": [
    "import opendatasets as od\n",
    "import pandas\n",
    "  \n",
    "od.download(\n",
    "    \"https://www.kaggle.com/gti-upm/leapgestrecog\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "pJdgs6RU5Kmg"
   },
   "source": [
    "Подключим все необходимые библиотеки:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "gYm-NdBx5s3L"
   },
   "outputs": [],
   "source": [
    "import os\n",
    "import torch\n",
    "import torchvision\n",
    "import torch.nn as nn\n",
    "import numpy as np\n",
    "import torch.nn.functional as F\n",
    "from torchvision.datasets import ImageFolder\n",
    "from torchvision import models\n",
    "from torch.utils.data import DataLoader, ConcatDataset\n",
    "import torchvision.transforms as tt\n",
    "import torchvision.transforms.functional as TF\n",
    "from torchvision.utils import make_grid\n",
    "from torchsummary import summary\n",
    "import matplotlib.pyplot as plt\n",
    "%matplotlib inline\n",
    "import time\n",
    "from torch.autograd import Variable\n",
    "import cv2\n",
    "from PIL import Image"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "axb5vaeYbe-2"
   },
   "outputs": [],
   "source": [
    "DATA_ROOT = 'leapgestrecog/leapGestRecog'"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "eIFpsXps5UF1"
   },
   "source": [
    "Зададим трансформации для трейна и теста. Для Grayscale задаем число выходных слоев, равное 3, т.к. resnet не принимает одноканальные изображения"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "5oj8aMNwb1ls"
   },
   "outputs": [],
   "source": [
    "train_tfms = tt.Compose([\n",
    "                         tt.Grayscale(num_output_channels=3), # Картинки черно-белые\n",
    "                         tt.Resize([128, 128]),\n",
    "                         # Настройки для расширения датасета\n",
    "                         tt.RandomHorizontalFlip(),           # Случайные повороты на 90 градусов\n",
    "                         tt.RandomRotation(30),               # Случайные повороты на 30 градусов\n",
    "                         tt.ToTensor(),                       # Приведение к тензору\n",
    "                         ])                      \n",
    "\n",
    "test_tfms = tt.Compose([\n",
    "                        tt.Grayscale(num_output_channels=3),\n",
    "                        tt.Resize([128, 128]),\n",
    "                        tt.ToTensor(),\n",
    "                        ])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "F1LxX9u65b--"
   },
   "source": [
    "Сохраним имена классов через загрузку одной из папок:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "AxwRHMQzhKAX",
    "outputId": "af7a2209-778a-476a-ecf6-bf0a8d9b8d84"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['01_palm',\n",
       " '02_l',\n",
       " '03_fist',\n",
       " '04_fist_moved',\n",
       " '05_thumb',\n",
       " '06_index',\n",
       " '07_ok',\n",
       " '08_palm_moved',\n",
       " '09_c',\n",
       " '10_down']"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "tmp_ds = ImageFolder(os.path.join(DATA_ROOT, '00'))\n",
    "CLASSES_NAME = tmp_ds.classes\n",
    "CLASSES_NAME"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "cEXldWZY5k1e"
   },
   "source": [
    "Создадим тренировочный и тестовый датасеты с одними и теми же картинками, но с разным препроцессингом:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "gO2xjZnjeU91"
   },
   "outputs": [],
   "source": [
    "train_ds_list = []\n",
    "test_ds_list = []\n",
    "for dir in os.listdir(DATA_ROOT):\n",
    "  train_ds = ImageFolder(os.path.join(DATA_ROOT, dir), train_tfms)\n",
    "  test_ds = ImageFolder(os.path.join(DATA_ROOT, dir), test_tfms)\n",
    "  train_ds_list.append(train_ds)\n",
    "  test_ds_list.append(test_ds)\n",
    "\n",
    "train_ds = ConcatDataset(train_ds_list)\n",
    "test_ds = ConcatDataset(test_ds_list)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "PhpxGqSd52JI"
   },
   "source": [
    "Общее количество картинок:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "LpN_sgoGL0OI",
    "outputId": "b2b3dcfa-abb2-4c3d-942a-46312921048b"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "20000"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "LEN_DS = len(train_ds)\n",
    "LEN_DS"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "FnFrOVvO55f9"
   },
   "source": [
    "Функция преобразования номера класса в его название:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "1t0YkNg7kJNW"
   },
   "outputs": [],
   "source": [
    "def digit_to_classname(digit):\n",
    "  for idx, itm in enumerate(CLASSES_NAME):\n",
    "    if idx == digit:\n",
    "      return itm"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "vQMkAiVZ6Ces"
   },
   "source": [
    "Посмотрим распределения классов в датасете:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "5QsZ8zH7ZQl0",
    "outputId": "63af7293-75a7-49c7-976c-e097853f75c5"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "{'01_palm': 2000, '02_l': 2000, '03_fist': 2000, '04_fist_moved': 2000, '05_thumb': 2000, '06_index': 2000, '07_ok': 2000, '08_palm_moved': 2000, '09_c': 2000, '10_down': 2000}\n"
     ]
    }
   ],
   "source": [
    "from collections import Counter\n",
    "tmp_list = []\n",
    "for _, cls in train_ds:\n",
    "  tmp_list.append(digit_to_classname(cls))\n",
    "cnt = Counter(tmp_list)\n",
    "print(dict(cnt))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "gyHAnWqG6IH0"
   },
   "source": [
    "Распределение классов равное.\n",
    "\n",
    "Разделим датасет на тренировачный и тестовый:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "Zm_0uXCJNRch",
    "outputId": "0676071e-350c-42be-e45c-018cf48d17f2"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "(16000, 4000)"
      ]
     },
     "execution_count": 11,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "torch.manual_seed(1)\n",
    "\n",
    "val_split = 0.2\n",
    "split = int(LEN_DS * val_split)\n",
    "indices = torch.randperm(LEN_DS)\n",
    "\n",
    "train_ds = torch.utils.data.Subset(train_ds, indices[split:])\n",
    "test_ds = torch.utils.data.Subset(test_ds, indices[:split])\n",
    "\n",
    "len(train_ds), len(test_ds)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "N1EosScf6SU-"
   },
   "source": [
    "Проверим размерность случайного тензора в датасете:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "jyaVmZpu2yD8",
    "outputId": "84859516-e473-4bf9-afff-fdc4b6843863"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "torch.Size([3, 128, 128])"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "train_ds[15000][0].size()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "_gt_QweW6Z9H"
   },
   "source": [
    "Проверим дисбаланс классов:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "PmEVYt0C2rbv",
    "outputId": "f9156876-f074-487a-ce85-812b39cceecf"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "{'05_thumb': 1608, '02_l': 1609, '01_palm': 1587, '06_index': 1606, '04_fist_moved': 1610, '03_fist': 1590, '09_c': 1610, '07_ok': 1594, '08_palm_moved': 1594, '10_down': 1592}\n"
     ]
    }
   ],
   "source": [
    "from collections import Counter\n",
    "tmp_list = []\n",
    "for _, cls in train_ds:\n",
    "  tmp_list.append(digit_to_classname(cls))\n",
    "cnt = Counter(tmp_list)\n",
    "print(dict(cnt))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "AmAvgea06hAU"
   },
   "source": [
    "Дисбаланс классов есть, но он очень не существенный\n",
    "\n",
    "Создадим даталоадеры:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "uBFvRurbm4Dq"
   },
   "outputs": [],
   "source": [
    "batch_size = 64\n",
    "train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2)\n",
    "test_dl = DataLoader(test_ds, batch_size, num_workers=1, shuffle=False)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "N6FIAiUU6ta4"
   },
   "source": [
    "Выведем случайное изображение в тестовом загрузчике:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 305
    },
    "id": "xNUQG0rlnkYc",
    "outputId": "f7f828d0-f717-4f72-db9c-fcea8be39c9a"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "torch.Size([3, 128, 128])\n",
      "tensor(4) 05_thumb\n"
     ]
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "for img, l in train_dl:\n",
    "    print(img[0].shape)\n",
    "    print(l[0], digit_to_classname(l[0]))\n",
    "    plt.imshow(img[0].permute(1, 2, 0)) #, cmap='gray')\n",
    "    break"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "OyiLdPHw64Gm"
   },
   "source": [
    "Функция для визуалиции батча:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "6tsV-cvVNT3N"
   },
   "outputs": [],
   "source": [
    "def show_batch(dl):\n",
    "    for images, labels in dl:\n",
    "        fig, ax = plt.subplots(figsize=(12, 12))\n",
    "        ax.set_xticks([]); ax.set_yticks([])\n",
    "        print(images[0].shape)\n",
    "        ax.imshow(make_grid(images[:64], nrow=8).permute(1, 2, 0))\n",
    "        break"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "tBiXti1r6-E_"
   },
   "source": [
    "Визуализируем батч:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 705
    },
    "id": "8qHBmKwuabkA",
    "outputId": "0a521477-61dd-48c8-9d16-17575e4da13a"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "torch.Size([3, 128, 128])\n"
     ]
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 864x864 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "show_batch(train_dl)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "MO2CZqQ07DWu"
   },
   "source": [
    "Определим устройство для работы нейросетей:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 36
    },
    "id": "YIoDLZ8LZRKj",
    "outputId": "61cd43c0-d0dc-45b8-ec29-976aa92790bc"
   },
   "outputs": [
    {
     "data": {
      "application/vnd.google.colaboratory.intrinsic+json": {
       "type": "string"
      },
      "text/plain": [
       "'cuda'"
      ]
     },
     "execution_count": 18,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "device = 'cuda' if torch.cuda.is_available() else 'cpu'\n",
    "device"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "aoRgMw1v7KJ4"
   },
   "source": [
    "Возьмем для классификации предобученную сеть resnet50:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 1000,
     "referenced_widgets": [
      "79ad86576b824112b04c7b993b96fd7a",
      "e34eb8bc9d444d14b9a31cd8537ee725",
      "658e31dcb92542eb8ea75acb82c8acec",
      "094c7a001c1049c786160cb67562712c",
      "718db746d0e6410d9ad5223df0ca300e",
      "145a4210965b4cb182df9e65c6a83838",
      "c99e49879b194cb4b62b8fb589cd942b",
      "ab85f01ff9f646d39c86bd389f4cc7cf",
      "831660235cd4450e9e92f09cfdb0d9bc",
      "7267bdf841bf41acb9cd60eefe3e9250",
      "cd83185ca5a64d1c86c8e296a9ba0275"
     ]
    },
    "id": "6LuWg60rYkZJ",
    "outputId": "e922bd1c-9f27-4b24-8885-508cacdf1d0f"
   },
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.\n",
      "  warnings.warn(\n",
      "/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.\n",
      "  warnings.warn(msg)\n",
      "Downloading: \"https://download.pytorch.org/models/resnet50-0676ba61.pth\" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth\n"
     ]
    },
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "79ad86576b824112b04c7b993b96fd7a",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "  0%|          | 0.00/97.8M [00:00<?, ?B/s]"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "----------------------------------------------------------------\n",
      "        Layer (type)               Output Shape         Param #\n",
      "================================================================\n",
      "            Conv2d-1           [-1, 64, 64, 64]           9,408\n",
      "       BatchNorm2d-2           [-1, 64, 64, 64]             128\n",
      "              ReLU-3           [-1, 64, 64, 64]               0\n",
      "         MaxPool2d-4           [-1, 64, 32, 32]               0\n",
      "            Conv2d-5           [-1, 64, 32, 32]           4,096\n",
      "       BatchNorm2d-6           [-1, 64, 32, 32]             128\n",
      "              ReLU-7           [-1, 64, 32, 32]               0\n",
      "            Conv2d-8           [-1, 64, 32, 32]          36,864\n",
      "       BatchNorm2d-9           [-1, 64, 32, 32]             128\n",
      "             ReLU-10           [-1, 64, 32, 32]               0\n",
      "           Conv2d-11          [-1, 256, 32, 32]          16,384\n",
      "      BatchNorm2d-12          [-1, 256, 32, 32]             512\n",
      "           Conv2d-13          [-1, 256, 32, 32]          16,384\n",
      "      BatchNorm2d-14          [-1, 256, 32, 32]             512\n",
      "             ReLU-15          [-1, 256, 32, 32]               0\n",
      "       Bottleneck-16          [-1, 256, 32, 32]               0\n",
      "           Conv2d-17           [-1, 64, 32, 32]          16,384\n",
      "      BatchNorm2d-18           [-1, 64, 32, 32]             128\n",
      "             ReLU-19           [-1, 64, 32, 32]               0\n",
      "           Conv2d-20           [-1, 64, 32, 32]          36,864\n",
      "      BatchNorm2d-21           [-1, 64, 32, 32]             128\n",
      "             ReLU-22           [-1, 64, 32, 32]               0\n",
      "           Conv2d-23          [-1, 256, 32, 32]          16,384\n",
      "      BatchNorm2d-24          [-1, 256, 32, 32]             512\n",
      "             ReLU-25          [-1, 256, 32, 32]               0\n",
      "       Bottleneck-26          [-1, 256, 32, 32]               0\n",
      "           Conv2d-27           [-1, 64, 32, 32]          16,384\n",
      "      BatchNorm2d-28           [-1, 64, 32, 32]             128\n",
      "             ReLU-29           [-1, 64, 32, 32]               0\n",
      "           Conv2d-30           [-1, 64, 32, 32]          36,864\n",
      "      BatchNorm2d-31           [-1, 64, 32, 32]             128\n",
      "             ReLU-32           [-1, 64, 32, 32]               0\n",
      "           Conv2d-33          [-1, 256, 32, 32]          16,384\n",
      "      BatchNorm2d-34          [-1, 256, 32, 32]             512\n",
      "             ReLU-35          [-1, 256, 32, 32]               0\n",
      "       Bottleneck-36          [-1, 256, 32, 32]               0\n",
      "           Conv2d-37          [-1, 128, 32, 32]          32,768\n",
      "      BatchNorm2d-38          [-1, 128, 32, 32]             256\n",
      "             ReLU-39          [-1, 128, 32, 32]               0\n",
      "           Conv2d-40          [-1, 128, 16, 16]         147,456\n",
      "      BatchNorm2d-41          [-1, 128, 16, 16]             256\n",
      "             ReLU-42          [-1, 128, 16, 16]               0\n",
      "           Conv2d-43          [-1, 512, 16, 16]          65,536\n",
      "      BatchNorm2d-44          [-1, 512, 16, 16]           1,024\n",
      "           Conv2d-45          [-1, 512, 16, 16]         131,072\n",
      "      BatchNorm2d-46          [-1, 512, 16, 16]           1,024\n",
      "             ReLU-47          [-1, 512, 16, 16]               0\n",
      "       Bottleneck-48          [-1, 512, 16, 16]               0\n",
      "           Conv2d-49          [-1, 128, 16, 16]          65,536\n",
      "      BatchNorm2d-50          [-1, 128, 16, 16]             256\n",
      "             ReLU-51          [-1, 128, 16, 16]               0\n",
      "           Conv2d-52          [-1, 128, 16, 16]         147,456\n",
      "      BatchNorm2d-53          [-1, 128, 16, 16]             256\n",
      "             ReLU-54          [-1, 128, 16, 16]               0\n",
      "           Conv2d-55          [-1, 512, 16, 16]          65,536\n",
      "      BatchNorm2d-56          [-1, 512, 16, 16]           1,024\n",
      "             ReLU-57          [-1, 512, 16, 16]               0\n",
      "       Bottleneck-58          [-1, 512, 16, 16]               0\n",
      "           Conv2d-59          [-1, 128, 16, 16]          65,536\n",
      "      BatchNorm2d-60          [-1, 128, 16, 16]             256\n",
      "             ReLU-61          [-1, 128, 16, 16]               0\n",
      "           Conv2d-62          [-1, 128, 16, 16]         147,456\n",
      "      BatchNorm2d-63          [-1, 128, 16, 16]             256\n",
      "             ReLU-64          [-1, 128, 16, 16]               0\n",
      "           Conv2d-65          [-1, 512, 16, 16]          65,536\n",
      "      BatchNorm2d-66          [-1, 512, 16, 16]           1,024\n",
      "             ReLU-67          [-1, 512, 16, 16]               0\n",
      "       Bottleneck-68          [-1, 512, 16, 16]               0\n",
      "           Conv2d-69          [-1, 128, 16, 16]          65,536\n",
      "      BatchNorm2d-70          [-1, 128, 16, 16]             256\n",
      "             ReLU-71          [-1, 128, 16, 16]               0\n",
      "           Conv2d-72          [-1, 128, 16, 16]         147,456\n",
      "      BatchNorm2d-73          [-1, 128, 16, 16]             256\n",
      "             ReLU-74          [-1, 128, 16, 16]               0\n",
      "           Conv2d-75          [-1, 512, 16, 16]          65,536\n",
      "      BatchNorm2d-76          [-1, 512, 16, 16]           1,024\n",
      "             ReLU-77          [-1, 512, 16, 16]               0\n",
      "       Bottleneck-78          [-1, 512, 16, 16]               0\n",
      "           Conv2d-79          [-1, 256, 16, 16]         131,072\n",
      "      BatchNorm2d-80          [-1, 256, 16, 16]             512\n",
      "             ReLU-81          [-1, 256, 16, 16]               0\n",
      "           Conv2d-82            [-1, 256, 8, 8]         589,824\n",
      "      BatchNorm2d-83            [-1, 256, 8, 8]             512\n",
      "             ReLU-84            [-1, 256, 8, 8]               0\n",
      "           Conv2d-85           [-1, 1024, 8, 8]         262,144\n",
      "      BatchNorm2d-86           [-1, 1024, 8, 8]           2,048\n",
      "           Conv2d-87           [-1, 1024, 8, 8]         524,288\n",
      "      BatchNorm2d-88           [-1, 1024, 8, 8]           2,048\n",
      "             ReLU-89           [-1, 1024, 8, 8]               0\n",
      "       Bottleneck-90           [-1, 1024, 8, 8]               0\n",
      "           Conv2d-91            [-1, 256, 8, 8]         262,144\n",
      "      BatchNorm2d-92            [-1, 256, 8, 8]             512\n",
      "             ReLU-93            [-1, 256, 8, 8]               0\n",
      "           Conv2d-94            [-1, 256, 8, 8]         589,824\n",
      "      BatchNorm2d-95            [-1, 256, 8, 8]             512\n",
      "             ReLU-96            [-1, 256, 8, 8]               0\n",
      "           Conv2d-97           [-1, 1024, 8, 8]         262,144\n",
      "      BatchNorm2d-98           [-1, 1024, 8, 8]           2,048\n",
      "             ReLU-99           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-100           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-101            [-1, 256, 8, 8]         262,144\n",
      "     BatchNorm2d-102            [-1, 256, 8, 8]             512\n",
      "            ReLU-103            [-1, 256, 8, 8]               0\n",
      "          Conv2d-104            [-1, 256, 8, 8]         589,824\n",
      "     BatchNorm2d-105            [-1, 256, 8, 8]             512\n",
      "            ReLU-106            [-1, 256, 8, 8]               0\n",
      "          Conv2d-107           [-1, 1024, 8, 8]         262,144\n",
      "     BatchNorm2d-108           [-1, 1024, 8, 8]           2,048\n",
      "            ReLU-109           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-110           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-111            [-1, 256, 8, 8]         262,144\n",
      "     BatchNorm2d-112            [-1, 256, 8, 8]             512\n",
      "            ReLU-113            [-1, 256, 8, 8]               0\n",
      "          Conv2d-114            [-1, 256, 8, 8]         589,824\n",
      "     BatchNorm2d-115            [-1, 256, 8, 8]             512\n",
      "            ReLU-116            [-1, 256, 8, 8]               0\n",
      "          Conv2d-117           [-1, 1024, 8, 8]         262,144\n",
      "     BatchNorm2d-118           [-1, 1024, 8, 8]           2,048\n",
      "            ReLU-119           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-120           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-121            [-1, 256, 8, 8]         262,144\n",
      "     BatchNorm2d-122            [-1, 256, 8, 8]             512\n",
      "            ReLU-123            [-1, 256, 8, 8]               0\n",
      "          Conv2d-124            [-1, 256, 8, 8]         589,824\n",
      "     BatchNorm2d-125            [-1, 256, 8, 8]             512\n",
      "            ReLU-126            [-1, 256, 8, 8]               0\n",
      "          Conv2d-127           [-1, 1024, 8, 8]         262,144\n",
      "     BatchNorm2d-128           [-1, 1024, 8, 8]           2,048\n",
      "            ReLU-129           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-130           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-131            [-1, 256, 8, 8]         262,144\n",
      "     BatchNorm2d-132            [-1, 256, 8, 8]             512\n",
      "            ReLU-133            [-1, 256, 8, 8]               0\n",
      "          Conv2d-134            [-1, 256, 8, 8]         589,824\n",
      "     BatchNorm2d-135            [-1, 256, 8, 8]             512\n",
      "            ReLU-136            [-1, 256, 8, 8]               0\n",
      "          Conv2d-137           [-1, 1024, 8, 8]         262,144\n",
      "     BatchNorm2d-138           [-1, 1024, 8, 8]           2,048\n",
      "            ReLU-139           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-140           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-141            [-1, 512, 8, 8]         524,288\n",
      "     BatchNorm2d-142            [-1, 512, 8, 8]           1,024\n",
      "            ReLU-143            [-1, 512, 8, 8]               0\n",
      "          Conv2d-144            [-1, 512, 4, 4]       2,359,296\n",
      "     BatchNorm2d-145            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-146            [-1, 512, 4, 4]               0\n",
      "          Conv2d-147           [-1, 2048, 4, 4]       1,048,576\n",
      "     BatchNorm2d-148           [-1, 2048, 4, 4]           4,096\n",
      "          Conv2d-149           [-1, 2048, 4, 4]       2,097,152\n",
      "     BatchNorm2d-150           [-1, 2048, 4, 4]           4,096\n",
      "            ReLU-151           [-1, 2048, 4, 4]               0\n",
      "      Bottleneck-152           [-1, 2048, 4, 4]               0\n",
      "          Conv2d-153            [-1, 512, 4, 4]       1,048,576\n",
      "     BatchNorm2d-154            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-155            [-1, 512, 4, 4]               0\n",
      "          Conv2d-156            [-1, 512, 4, 4]       2,359,296\n",
      "     BatchNorm2d-157            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-158            [-1, 512, 4, 4]               0\n",
      "          Conv2d-159           [-1, 2048, 4, 4]       1,048,576\n",
      "     BatchNorm2d-160           [-1, 2048, 4, 4]           4,096\n",
      "            ReLU-161           [-1, 2048, 4, 4]               0\n",
      "      Bottleneck-162           [-1, 2048, 4, 4]               0\n",
      "          Conv2d-163            [-1, 512, 4, 4]       1,048,576\n",
      "     BatchNorm2d-164            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-165            [-1, 512, 4, 4]               0\n",
      "          Conv2d-166            [-1, 512, 4, 4]       2,359,296\n",
      "     BatchNorm2d-167            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-168            [-1, 512, 4, 4]               0\n",
      "          Conv2d-169           [-1, 2048, 4, 4]       1,048,576\n",
      "     BatchNorm2d-170           [-1, 2048, 4, 4]           4,096\n",
      "            ReLU-171           [-1, 2048, 4, 4]               0\n",
      "      Bottleneck-172           [-1, 2048, 4, 4]               0\n",
      "AdaptiveAvgPool2d-173           [-1, 2048, 1, 1]               0\n",
      "          Linear-174                 [-1, 1000]       2,049,000\n",
      "================================================================\n",
      "Total params: 25,557,032\n",
      "Trainable params: 25,557,032\n",
      "Non-trainable params: 0\n",
      "----------------------------------------------------------------\n",
      "Input size (MB): 0.19\n",
      "Forward/backward pass size (MB): 93.59\n",
      "Params size (MB): 97.49\n",
      "Estimated Total Size (MB): 191.27\n",
      "----------------------------------------------------------------\n"
     ]
    }
   ],
   "source": [
    "resnet = models.resnet50(pretrained=True)\n",
    "summary(resnet.to(device), input_size=(3, 128, 128))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "p4D5L6Nr70g4"
   },
   "source": [
    "Запретим весам обучаться"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "BO2tPEyfZxny"
   },
   "outputs": [],
   "source": [
    "for param in resnet.parameters():\n",
    "    param.requires_grad = False"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "UHp4xoxL79yz"
   },
   "source": [
    "Посмотрим на слои нейросети. Нас интересует последний классификационный слой, который мы будем переобучать:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "J9g8OqwicT9v",
    "outputId": "8df28540-e889-4b4e-ee37-8a0dc2635a37"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "ResNet(\n",
       "  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)\n",
       "  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "  (relu): ReLU(inplace=True)\n",
       "  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)\n",
       "  (layer1): Sequential(\n",
       "    (0): Bottleneck(\n",
       "      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "      (downsample): Sequential(\n",
       "        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      )\n",
       "    )\n",
       "    (1): Bottleneck(\n",
       "      (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (2): Bottleneck(\n",
       "      (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "  )\n",
       "  (layer2): Sequential(\n",
       "    (0): Bottleneck(\n",
       "      (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "      (downsample): Sequential(\n",
       "        (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)\n",
       "        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      )\n",
       "    )\n",
       "    (1): Bottleneck(\n",
       "      (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (2): Bottleneck(\n",
       "      (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (3): Bottleneck(\n",
       "      (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "  )\n",
       "  (layer3): Sequential(\n",
       "    (0): Bottleneck(\n",
       "      (conv1): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "      (downsample): Sequential(\n",
       "        (0): Conv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), bias=False)\n",
       "        (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      )\n",
       "    )\n",
       "    (1): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (2): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (3): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (4): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (5): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "  )\n",
       "  (layer4): Sequential(\n",
       "    (0): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "      (downsample): Sequential(\n",
       "        (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)\n",
       "        (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      )\n",
       "    )\n",
       "    (1): Bottleneck(\n",
       "      (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (2): Bottleneck(\n",
       "      (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "  )\n",
       "  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))\n",
       "  (fc): Linear(in_features=2048, out_features=1000, bias=True)\n",
       ")"
      ]
     },
     "execution_count": 21,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "resnet"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "fjNDTYXn8LKK"
   },
   "source": [
    "Перезапишем последний классификационный слой, и убедимся, что остальные слои не будут обучаться:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "bdPv8w9CcVpl",
    "outputId": "a294ea2d-761a-4779-d5b4-1f6939fb6dfc"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "----------------------------------------------------------------\n",
      "        Layer (type)               Output Shape         Param #\n",
      "================================================================\n",
      "            Conv2d-1           [-1, 64, 64, 64]           9,408\n",
      "       BatchNorm2d-2           [-1, 64, 64, 64]             128\n",
      "              ReLU-3           [-1, 64, 64, 64]               0\n",
      "         MaxPool2d-4           [-1, 64, 32, 32]               0\n",
      "            Conv2d-5           [-1, 64, 32, 32]           4,096\n",
      "       BatchNorm2d-6           [-1, 64, 32, 32]             128\n",
      "              ReLU-7           [-1, 64, 32, 32]               0\n",
      "            Conv2d-8           [-1, 64, 32, 32]          36,864\n",
      "       BatchNorm2d-9           [-1, 64, 32, 32]             128\n",
      "             ReLU-10           [-1, 64, 32, 32]               0\n",
      "           Conv2d-11          [-1, 256, 32, 32]          16,384\n",
      "      BatchNorm2d-12          [-1, 256, 32, 32]             512\n",
      "           Conv2d-13          [-1, 256, 32, 32]          16,384\n",
      "      BatchNorm2d-14          [-1, 256, 32, 32]             512\n",
      "             ReLU-15          [-1, 256, 32, 32]               0\n",
      "       Bottleneck-16          [-1, 256, 32, 32]               0\n",
      "           Conv2d-17           [-1, 64, 32, 32]          16,384\n",
      "      BatchNorm2d-18           [-1, 64, 32, 32]             128\n",
      "             ReLU-19           [-1, 64, 32, 32]               0\n",
      "           Conv2d-20           [-1, 64, 32, 32]          36,864\n",
      "      BatchNorm2d-21           [-1, 64, 32, 32]             128\n",
      "             ReLU-22           [-1, 64, 32, 32]               0\n",
      "           Conv2d-23          [-1, 256, 32, 32]          16,384\n",
      "      BatchNorm2d-24          [-1, 256, 32, 32]             512\n",
      "             ReLU-25          [-1, 256, 32, 32]               0\n",
      "       Bottleneck-26          [-1, 256, 32, 32]               0\n",
      "           Conv2d-27           [-1, 64, 32, 32]          16,384\n",
      "      BatchNorm2d-28           [-1, 64, 32, 32]             128\n",
      "             ReLU-29           [-1, 64, 32, 32]               0\n",
      "           Conv2d-30           [-1, 64, 32, 32]          36,864\n",
      "      BatchNorm2d-31           [-1, 64, 32, 32]             128\n",
      "             ReLU-32           [-1, 64, 32, 32]               0\n",
      "           Conv2d-33          [-1, 256, 32, 32]          16,384\n",
      "      BatchNorm2d-34          [-1, 256, 32, 32]             512\n",
      "             ReLU-35          [-1, 256, 32, 32]               0\n",
      "       Bottleneck-36          [-1, 256, 32, 32]               0\n",
      "           Conv2d-37          [-1, 128, 32, 32]          32,768\n",
      "      BatchNorm2d-38          [-1, 128, 32, 32]             256\n",
      "             ReLU-39          [-1, 128, 32, 32]               0\n",
      "           Conv2d-40          [-1, 128, 16, 16]         147,456\n",
      "      BatchNorm2d-41          [-1, 128, 16, 16]             256\n",
      "             ReLU-42          [-1, 128, 16, 16]               0\n",
      "           Conv2d-43          [-1, 512, 16, 16]          65,536\n",
      "      BatchNorm2d-44          [-1, 512, 16, 16]           1,024\n",
      "           Conv2d-45          [-1, 512, 16, 16]         131,072\n",
      "      BatchNorm2d-46          [-1, 512, 16, 16]           1,024\n",
      "             ReLU-47          [-1, 512, 16, 16]               0\n",
      "       Bottleneck-48          [-1, 512, 16, 16]               0\n",
      "           Conv2d-49          [-1, 128, 16, 16]          65,536\n",
      "      BatchNorm2d-50          [-1, 128, 16, 16]             256\n",
      "             ReLU-51          [-1, 128, 16, 16]               0\n",
      "           Conv2d-52          [-1, 128, 16, 16]         147,456\n",
      "      BatchNorm2d-53          [-1, 128, 16, 16]             256\n",
      "             ReLU-54          [-1, 128, 16, 16]               0\n",
      "           Conv2d-55          [-1, 512, 16, 16]          65,536\n",
      "      BatchNorm2d-56          [-1, 512, 16, 16]           1,024\n",
      "             ReLU-57          [-1, 512, 16, 16]               0\n",
      "       Bottleneck-58          [-1, 512, 16, 16]               0\n",
      "           Conv2d-59          [-1, 128, 16, 16]          65,536\n",
      "      BatchNorm2d-60          [-1, 128, 16, 16]             256\n",
      "             ReLU-61          [-1, 128, 16, 16]               0\n",
      "           Conv2d-62          [-1, 128, 16, 16]         147,456\n",
      "      BatchNorm2d-63          [-1, 128, 16, 16]             256\n",
      "             ReLU-64          [-1, 128, 16, 16]               0\n",
      "           Conv2d-65          [-1, 512, 16, 16]          65,536\n",
      "      BatchNorm2d-66          [-1, 512, 16, 16]           1,024\n",
      "             ReLU-67          [-1, 512, 16, 16]               0\n",
      "       Bottleneck-68          [-1, 512, 16, 16]               0\n",
      "           Conv2d-69          [-1, 128, 16, 16]          65,536\n",
      "      BatchNorm2d-70          [-1, 128, 16, 16]             256\n",
      "             ReLU-71          [-1, 128, 16, 16]               0\n",
      "           Conv2d-72          [-1, 128, 16, 16]         147,456\n",
      "      BatchNorm2d-73          [-1, 128, 16, 16]             256\n",
      "             ReLU-74          [-1, 128, 16, 16]               0\n",
      "           Conv2d-75          [-1, 512, 16, 16]          65,536\n",
      "      BatchNorm2d-76          [-1, 512, 16, 16]           1,024\n",
      "             ReLU-77          [-1, 512, 16, 16]               0\n",
      "       Bottleneck-78          [-1, 512, 16, 16]               0\n",
      "           Conv2d-79          [-1, 256, 16, 16]         131,072\n",
      "      BatchNorm2d-80          [-1, 256, 16, 16]             512\n",
      "             ReLU-81          [-1, 256, 16, 16]               0\n",
      "           Conv2d-82            [-1, 256, 8, 8]         589,824\n",
      "      BatchNorm2d-83            [-1, 256, 8, 8]             512\n",
      "             ReLU-84            [-1, 256, 8, 8]               0\n",
      "           Conv2d-85           [-1, 1024, 8, 8]         262,144\n",
      "      BatchNorm2d-86           [-1, 1024, 8, 8]           2,048\n",
      "           Conv2d-87           [-1, 1024, 8, 8]         524,288\n",
      "      BatchNorm2d-88           [-1, 1024, 8, 8]           2,048\n",
      "             ReLU-89           [-1, 1024, 8, 8]               0\n",
      "       Bottleneck-90           [-1, 1024, 8, 8]               0\n",
      "           Conv2d-91            [-1, 256, 8, 8]         262,144\n",
      "      BatchNorm2d-92            [-1, 256, 8, 8]             512\n",
      "             ReLU-93            [-1, 256, 8, 8]               0\n",
      "           Conv2d-94            [-1, 256, 8, 8]         589,824\n",
      "      BatchNorm2d-95            [-1, 256, 8, 8]             512\n",
      "             ReLU-96            [-1, 256, 8, 8]               0\n",
      "           Conv2d-97           [-1, 1024, 8, 8]         262,144\n",
      "      BatchNorm2d-98           [-1, 1024, 8, 8]           2,048\n",
      "             ReLU-99           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-100           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-101            [-1, 256, 8, 8]         262,144\n",
      "     BatchNorm2d-102            [-1, 256, 8, 8]             512\n",
      "            ReLU-103            [-1, 256, 8, 8]               0\n",
      "          Conv2d-104            [-1, 256, 8, 8]         589,824\n",
      "     BatchNorm2d-105            [-1, 256, 8, 8]             512\n",
      "            ReLU-106            [-1, 256, 8, 8]               0\n",
      "          Conv2d-107           [-1, 1024, 8, 8]         262,144\n",
      "     BatchNorm2d-108           [-1, 1024, 8, 8]           2,048\n",
      "            ReLU-109           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-110           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-111            [-1, 256, 8, 8]         262,144\n",
      "     BatchNorm2d-112            [-1, 256, 8, 8]             512\n",
      "            ReLU-113            [-1, 256, 8, 8]               0\n",
      "          Conv2d-114            [-1, 256, 8, 8]         589,824\n",
      "     BatchNorm2d-115            [-1, 256, 8, 8]             512\n",
      "            ReLU-116            [-1, 256, 8, 8]               0\n",
      "          Conv2d-117           [-1, 1024, 8, 8]         262,144\n",
      "     BatchNorm2d-118           [-1, 1024, 8, 8]           2,048\n",
      "            ReLU-119           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-120           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-121            [-1, 256, 8, 8]         262,144\n",
      "     BatchNorm2d-122            [-1, 256, 8, 8]             512\n",
      "            ReLU-123            [-1, 256, 8, 8]               0\n",
      "          Conv2d-124            [-1, 256, 8, 8]         589,824\n",
      "     BatchNorm2d-125            [-1, 256, 8, 8]             512\n",
      "            ReLU-126            [-1, 256, 8, 8]               0\n",
      "          Conv2d-127           [-1, 1024, 8, 8]         262,144\n",
      "     BatchNorm2d-128           [-1, 1024, 8, 8]           2,048\n",
      "            ReLU-129           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-130           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-131            [-1, 256, 8, 8]         262,144\n",
      "     BatchNorm2d-132            [-1, 256, 8, 8]             512\n",
      "            ReLU-133            [-1, 256, 8, 8]               0\n",
      "          Conv2d-134            [-1, 256, 8, 8]         589,824\n",
      "     BatchNorm2d-135            [-1, 256, 8, 8]             512\n",
      "            ReLU-136            [-1, 256, 8, 8]               0\n",
      "          Conv2d-137           [-1, 1024, 8, 8]         262,144\n",
      "     BatchNorm2d-138           [-1, 1024, 8, 8]           2,048\n",
      "            ReLU-139           [-1, 1024, 8, 8]               0\n",
      "      Bottleneck-140           [-1, 1024, 8, 8]               0\n",
      "          Conv2d-141            [-1, 512, 8, 8]         524,288\n",
      "     BatchNorm2d-142            [-1, 512, 8, 8]           1,024\n",
      "            ReLU-143            [-1, 512, 8, 8]               0\n",
      "          Conv2d-144            [-1, 512, 4, 4]       2,359,296\n",
      "     BatchNorm2d-145            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-146            [-1, 512, 4, 4]               0\n",
      "          Conv2d-147           [-1, 2048, 4, 4]       1,048,576\n",
      "     BatchNorm2d-148           [-1, 2048, 4, 4]           4,096\n",
      "          Conv2d-149           [-1, 2048, 4, 4]       2,097,152\n",
      "     BatchNorm2d-150           [-1, 2048, 4, 4]           4,096\n",
      "            ReLU-151           [-1, 2048, 4, 4]               0\n",
      "      Bottleneck-152           [-1, 2048, 4, 4]               0\n",
      "          Conv2d-153            [-1, 512, 4, 4]       1,048,576\n",
      "     BatchNorm2d-154            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-155            [-1, 512, 4, 4]               0\n",
      "          Conv2d-156            [-1, 512, 4, 4]       2,359,296\n",
      "     BatchNorm2d-157            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-158            [-1, 512, 4, 4]               0\n",
      "          Conv2d-159           [-1, 2048, 4, 4]       1,048,576\n",
      "     BatchNorm2d-160           [-1, 2048, 4, 4]           4,096\n",
      "            ReLU-161           [-1, 2048, 4, 4]               0\n",
      "      Bottleneck-162           [-1, 2048, 4, 4]               0\n",
      "          Conv2d-163            [-1, 512, 4, 4]       1,048,576\n",
      "     BatchNorm2d-164            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-165            [-1, 512, 4, 4]               0\n",
      "          Conv2d-166            [-1, 512, 4, 4]       2,359,296\n",
      "     BatchNorm2d-167            [-1, 512, 4, 4]           1,024\n",
      "            ReLU-168            [-1, 512, 4, 4]               0\n",
      "          Conv2d-169           [-1, 2048, 4, 4]       1,048,576\n",
      "     BatchNorm2d-170           [-1, 2048, 4, 4]           4,096\n",
      "            ReLU-171           [-1, 2048, 4, 4]               0\n",
      "      Bottleneck-172           [-1, 2048, 4, 4]               0\n",
      "AdaptiveAvgPool2d-173           [-1, 2048, 1, 1]               0\n",
      "          Linear-174                   [-1, 10]          20,490\n",
      "================================================================\n",
      "Total params: 23,528,522\n",
      "Trainable params: 20,490\n",
      "Non-trainable params: 23,508,032\n",
      "----------------------------------------------------------------\n",
      "Input size (MB): 0.19\n",
      "Forward/backward pass size (MB): 93.58\n",
      "Params size (MB): 89.75\n",
      "Estimated Total Size (MB): 183.52\n",
      "----------------------------------------------------------------\n"
     ]
    }
   ],
   "source": [
    "in_features = resnet.fc.in_features\n",
    "fc = nn.Linear(in_features=in_features, out_features=len(CLASSES_NAME))\n",
    "resnet.fc = fc\n",
    "\n",
    "summary(resnet.to(device), input_size=(3, 128, 128))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "iXa4ou4r8e-q"
   },
   "source": [
    "Создадим список обучаемых параметров и подадим его в оптимизатор. Также инициализируем функцию потерь:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "N7HRNeASc0VI"
   },
   "outputs": [],
   "source": [
    "params_to_update = []\n",
    "for name, param in resnet.named_parameters():\n",
    "    if param.requires_grad == True:\n",
    "        params_to_update.append(param)\n",
    "\n",
    "        \n",
    "criterion = nn.CrossEntropyLoss()\n",
    "optimizer = torch.optim.Adam(params_to_update, lr=0.001)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "ZNH5pUaT8uXm"
   },
   "source": [
    "Инициализируем функцию для обучения сети:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "KxNckRQGdeN9"
   },
   "outputs": [],
   "source": [
    "from time import time\n",
    "# from tqdm import tqdm\n",
    "\n",
    "def train(model,\n",
    "          criterion,\n",
    "          optimizer,\n",
    "          train_dataloader,\n",
    "          test_dataloader,\n",
    "          print_every,\n",
    "          num_epoch):\n",
    "    steps = 0\n",
    "    train_losses, val_losses = [], []\n",
    "\n",
    "    model.to(device)\n",
    "    for epoch in range(num_epoch):\n",
    "        running_loss = 0\n",
    "        correct_train = 0\n",
    "        total_train = 0\n",
    "        start_time = time()\n",
    "        iter_time = time()\n",
    "        \n",
    "        model.train()\n",
    "        for i, (images, labels) in enumerate(train_dataloader):\n",
    "            steps += 1\n",
    "            images = images.to(device)\n",
    "            labels = labels.to(device)\n",
    "\n",
    "            # Forward pass\n",
    "            output = model(images)\n",
    "            loss = criterion(output, labels)\n",
    "\n",
    "            correct_train += (torch.max(output, dim=1)[1] == labels).sum()\n",
    "            total_train += labels.size(0)\n",
    "\n",
    "            # Backward and optimize\n",
    "            optimizer.zero_grad()\n",
    "            loss.backward()\n",
    "            optimizer.step()\n",
    "\n",
    "            running_loss += loss.item()\n",
    "\n",
    "            # Logging\n",
    "            if steps % print_every == 0:\n",
    "                print(f'Epoch [{epoch + 1}]/[{num_epoch}]. Batch [{i + 1}]/[{len(train_dataloader)}].', end=' ')\n",
    "                print(f'Train loss {running_loss / steps:.3f}.', end=' ')\n",
    "                print(f'Train acc {correct_train / total_train * 100:.3f}.', end=' ')\n",
    "                with torch.no_grad():\n",
    "                    model.eval()\n",
    "                    correct_val, total_val = 0, 0\n",
    "                    val_loss = 0\n",
    "                    for images, labels in test_dataloader:\n",
    "                        images = images.to(device)\n",
    "                        labels = labels.to(device)\n",
    "                        output = model(images)\n",
    "                        loss = criterion(output, labels)\n",
    "                        val_loss += loss.item()\n",
    "\n",
    "                        correct_val += (torch.max(output, dim=1)[1] == labels).sum()\n",
    "                        total_val += labels.size(0)\n",
    "\n",
    "                print(f'Val loss {val_loss / len(test_dataloader):.3f}. Val acc {correct_val / total_val * 100:.3f}.', end=' ')\n",
    "                print(f'Took {time() - iter_time:.3f} seconds')\n",
    "                iter_time = time()\n",
    "\n",
    "                train_losses.append(running_loss / total_train)\n",
    "                val_losses.append(val_loss / total_val)\n",
    "\n",
    "\n",
    "        print(f'Epoch took {time() - start_time}') \n",
    "        torch.save(model, f'checkpoint_{correct_val / total_val * 100:.2f}')\n",
    "        \n",
    "    return model, train_losses, val_losses"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "GngA07Ll81Zp"
   },
   "source": [
    "Обучим сеть на 5 эпохах:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "QJfS91lnfYWh",
    "outputId": "0778b210-6a9e-415e-978e-341df76faafa"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Epoch [1]/[5]. Batch [50]/[250]. Train loss 1.304. Train acc 62.844. Val loss 1.339. Val acc 57.800. Took 30.066 seconds\n",
      "Epoch [1]/[5]. Batch [100]/[250]. Train loss 1.084. Train acc 69.391. Val loss 0.856. Val acc 77.250. Took 30.168 seconds\n",
      "Epoch [1]/[5]. Batch [150]/[250]. Train loss 0.922. Train acc 74.125. Val loss 0.719. Val acc 78.925. Took 33.065 seconds\n",
      "Epoch [1]/[5]. Batch [200]/[250]. Train loss 0.812. Train acc 77.164. Val loss 0.684. Val acc 79.625. Took 28.651 seconds\n",
      "Epoch [1]/[5]. Batch [250]/[250]. Train loss 0.737. Train acc 79.200. Val loss 0.590. Val acc 81.425. Took 27.826 seconds\n",
      "Epoch took 149.83549690246582\n",
      "Epoch [2]/[5]. Batch [50]/[250]. Train loss 0.106. Train acc 78.656. Val loss 0.709. Val acc 77.800. Took 30.622 seconds\n",
      "Epoch [2]/[5]. Batch [100]/[250]. Train loss 0.153. Train acc 81.969. Val loss 0.592. Val acc 83.325. Took 28.284 seconds\n",
      "Epoch [2]/[5]. Batch [150]/[250]. Train loss 0.183. Train acc 84.042. Val loss 0.559. Val acc 84.350. Took 28.849 seconds\n",
      "Epoch [2]/[5]. Batch [200]/[250]. Train loss 0.203. Train acc 85.352. Val loss 0.528. Val acc 85.150. Took 28.168 seconds\n",
      "Epoch [2]/[5]. Batch [250]/[250]. Train loss 0.216. Train acc 86.231. Val loss 0.548. Val acc 82.925. Took 27.519 seconds\n",
      "Epoch took 143.49886918067932\n",
      "Epoch [3]/[5]. Batch [50]/[250]. Train loss 0.034. Train acc 87.625. Val loss 0.560. Val acc 84.125. Took 29.457 seconds\n",
      "Epoch [3]/[5]. Batch [100]/[250]. Train loss 0.059. Train acc 88.828. Val loss 0.549. Val acc 82.600. Took 28.968 seconds\n",
      "Epoch [3]/[5]. Batch [150]/[250]. Train loss 0.077. Train acc 89.479. Val loss 0.472. Val acc 87.300. Took 28.158 seconds\n",
      "Epoch [3]/[5]. Batch [200]/[250]. Train loss 0.092. Train acc 90.031. Val loss 0.477. Val acc 86.000. Took 28.650 seconds\n",
      "Epoch [3]/[5]. Batch [250]/[250]. Train loss 0.104. Train acc 90.300. Val loss 0.436. Val acc 87.625. Took 27.853 seconds\n",
      "Epoch took 143.1436161994934\n",
      "Epoch [4]/[5]. Batch [50]/[250]. Train loss 0.018. Train acc 91.031. Val loss 0.445. Val acc 85.425. Took 30.160 seconds\n",
      "Epoch [4]/[5]. Batch [100]/[250]. Train loss 0.032. Train acc 91.484. Val loss 0.428. Val acc 87.000. Took 28.197 seconds\n",
      "Epoch [4]/[5]. Batch [150]/[250]. Train loss 0.045. Train acc 91.635. Val loss 0.399. Val acc 87.675. Took 28.191 seconds\n",
      "Epoch [4]/[5]. Batch [200]/[250]. Train loss 0.055. Train acc 91.914. Val loss 0.366. Val acc 90.500. Took 28.241 seconds\n",
      "Epoch [4]/[5]. Batch [250]/[250]. Train loss 0.064. Train acc 92.075. Val loss 0.373. Val acc 89.600. Took 28.497 seconds\n",
      "Epoch took 143.34473729133606\n",
      "Epoch [5]/[5]. Batch [50]/[250]. Train loss 0.013. Train acc 90.625. Val loss 0.375. Val acc 89.675. Took 29.248 seconds\n",
      "Epoch [5]/[5]. Batch [100]/[250]. Train loss 0.024. Train acc 91.547. Val loss 0.363. Val acc 89.775. Took 28.069 seconds\n",
      "Epoch [5]/[5]. Batch [150]/[250]. Train loss 0.032. Train acc 91.969. Val loss 0.366. Val acc 90.125. Took 28.111 seconds\n",
      "Epoch [5]/[5]. Batch [200]/[250]. Train loss 0.039. Train acc 92.516. Val loss 0.390. Val acc 87.850. Took 29.228 seconds\n",
      "Epoch [5]/[5]. Batch [250]/[250]. Train loss 0.046. Train acc 92.781. Val loss 0.363. Val acc 89.375. Took 27.322 seconds\n",
      "Epoch took 142.03448700904846\n"
     ]
    }
   ],
   "source": [
    "print_every = 50\n",
    "num_epoch = 5\n",
    "\n",
    "resnet, train_losses, val_losses = train(\n",
    "    model=resnet,\n",
    "    criterion=criterion,\n",
    "    optimizer=optimizer,\n",
    "    train_dataloader=train_dl,\n",
    "    test_dataloader=test_dl,\n",
    "    print_every=print_every,\n",
    "    num_epoch=num_epoch\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "qiaFki7-Dcur",
    "outputId": "2adce5c1-9a4b-4ec3-b4bc-965c93b2760c"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Epoch [1]/[1]. Batch [50]/[250]. Train loss 0.273. Train acc 90.406. Val loss 0.358. Val acc 90.450. Took 29.141 seconds\n",
      "Epoch [1]/[1]. Batch [100]/[250]. Train loss 0.252. Train acc 91.359. Val loss 0.351. Val acc 90.400. Took 28.167 seconds\n",
      "Epoch [1]/[1]. Batch [150]/[250]. Train loss 0.236. Train acc 92.031. Val loss 0.336. Val acc 91.400. Took 28.925 seconds\n",
      "Epoch [1]/[1]. Batch [200]/[250]. Train loss 0.229. Train acc 92.352. Val loss 0.336. Val acc 90.200. Took 28.320 seconds\n",
      "Epoch [1]/[1]. Batch [250]/[250]. Train loss 0.223. Train acc 92.581. Val loss 0.376. Val acc 88.250. Took 27.448 seconds\n",
      "Epoch took 142.05972576141357\n"
     ]
    }
   ],
   "source": [
    "resnet, train_losses, val_losses = train(\n",
    "    model=resnet,\n",
    "    criterion=criterion,\n",
    "    optimizer=optimizer,\n",
    "    train_dataloader=train_dl,\n",
    "    test_dataloader=test_dl,\n",
    "    print_every=print_every,\n",
    "    num_epoch=1\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "fa-yVv4x9HLV"
   },
   "source": [
    "Сохраним и скачаем нашу модель:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "ksW4s6nio9cc",
    "outputId": "f5688301-2622-4552-98d9-d42f25008989"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Mounted at /content/drive\n"
     ]
    }
   ],
   "source": [
    "from google.colab import drive\n",
    "drive.mount('/content/drive')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "BA5yUsTzpIdi"
   },
   "outputs": [],
   "source": [
    "torch.save(resnet, '/content/drive/MyDrive/!!GeekBrains/31 Pythoch для разработки ИНС/!Пономарева/10 Распознавание лиц и эмоций/Курсовая/checkpoint_93.25')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "LbTa1W4ugNp7",
    "outputId": "8fc2dd63-45cd-4987-86c3-71b2c48f1e6f"
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "ResNet(\n",
       "  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)\n",
       "  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "  (relu): ReLU(inplace=True)\n",
       "  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)\n",
       "  (layer1): Sequential(\n",
       "    (0): Bottleneck(\n",
       "      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "      (downsample): Sequential(\n",
       "        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      )\n",
       "    )\n",
       "    (1): Bottleneck(\n",
       "      (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (2): Bottleneck(\n",
       "      (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "  )\n",
       "  (layer2): Sequential(\n",
       "    (0): Bottleneck(\n",
       "      (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "      (downsample): Sequential(\n",
       "        (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)\n",
       "        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      )\n",
       "    )\n",
       "    (1): Bottleneck(\n",
       "      (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (2): Bottleneck(\n",
       "      (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (3): Bottleneck(\n",
       "      (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "  )\n",
       "  (layer3): Sequential(\n",
       "    (0): Bottleneck(\n",
       "      (conv1): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "      (downsample): Sequential(\n",
       "        (0): Conv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), bias=False)\n",
       "        (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      )\n",
       "    )\n",
       "    (1): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (2): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (3): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (4): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (5): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "  )\n",
       "  (layer4): Sequential(\n",
       "    (0): Bottleneck(\n",
       "      (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "      (downsample): Sequential(\n",
       "        (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)\n",
       "        (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      )\n",
       "    )\n",
       "    (1): Bottleneck(\n",
       "      (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "    (2): Bottleneck(\n",
       "      (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n",
       "      (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)\n",
       "      (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n",
       "      (relu): ReLU(inplace=True)\n",
       "    )\n",
       "  )\n",
       "  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))\n",
       "  (fc): Linear(in_features=2048, out_features=10, bias=True)\n",
       ")"
      ]
     },
     "execution_count": 30,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "resnet = torch.load('/content/drive/MyDrive/!!GeekBrains/31 Pythoch для разработки ИНС/!Пономарева/10 Распознавание лиц и эмоций/Курсовая/checkpoint_93.25')\n",
    "resnet.eval()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "BYkjVjpb9b5g"
   },
   "source": [
    "Посмотрим предсказания по 5 картинкам из тестового датасета:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 1000
    },
    "id": "NUgFmnzQfI6z",
    "outputId": "2834cb9f-3281-48a6-d003-53babb3c5103"
   },
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "with torch.no_grad():\n",
    "    for i, data in enumerate(test_dl, 3):\n",
    "        images, labels = data[0].to(device), data[1].to(device)\n",
    "        \n",
    "        outputs = resnet(images)\n",
    "        plt.title(f'pred - {digit_to_classname(outputs[0].argmax())}, gt - {digit_to_classname(labels[0])}')\n",
    "        plt.imshow(images[0].cpu().permute(1, 2, 0), cmap='gray')\n",
    "        plt.show()\n",
    "        if i>5:\n",
    "            break"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "p3SchgxN9otl"
   },
   "source": [
    "Ошибок в этих примерах нет.\n",
    "\n",
    "Установим сеть для детекции лица, а также сеть от Google для детекции рук, лица и тела mediapipe (использовать будем только детектор рук)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "7UUrMGovjUA6",
    "outputId": "15764947-68e7-4060-87f1-dc6a57024533"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/\n",
      "Collecting facenet_pytorch\n",
      "  Downloading facenet_pytorch-2.5.2-py3-none-any.whl (1.9 MB)\n",
      "\u001b[2K     \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m1.9/1.9 MB\u001b[0m \u001b[31m31.9 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
      "\u001b[?25hRequirement already satisfied: requests in /usr/local/lib/python3.8/dist-packages (from facenet_pytorch) (2.25.1)\n",
      "Requirement already satisfied: pillow in /usr/local/lib/python3.8/dist-packages (from facenet_pytorch) (7.1.2)\n",
      "Requirement already satisfied: numpy in /usr/local/lib/python3.8/dist-packages (from facenet_pytorch) (1.21.6)\n",
      "Requirement already satisfied: torchvision in /usr/local/lib/python3.8/dist-packages (from facenet_pytorch) (0.14.1+cu116)\n",
      "Requirement already satisfied: chardet<5,>=3.0.2 in /usr/local/lib/python3.8/dist-packages (from requests->facenet_pytorch) (4.0.0)\n",
      "Requirement already satisfied: urllib3<1.27,>=1.21.1 in /usr/local/lib/python3.8/dist-packages (from requests->facenet_pytorch) (1.24.3)\n",
      "Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.8/dist-packages (from requests->facenet_pytorch) (2.10)\n",
      "Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.8/dist-packages (from requests->facenet_pytorch) (2022.12.7)\n",
      "Requirement already satisfied: torch==1.13.1 in /usr/local/lib/python3.8/dist-packages (from torchvision->facenet_pytorch) (1.13.1+cu116)\n",
      "Requirement already satisfied: typing-extensions in /usr/local/lib/python3.8/dist-packages (from torchvision->facenet_pytorch) (4.5.0)\n",
      "Installing collected packages: facenet_pytorch\n",
      "Successfully installed facenet_pytorch-2.5.2\n"
     ]
    }
   ],
   "source": [
    "!pip install facenet_pytorch"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "36DNSwNR75nl",
    "outputId": "82900529-d932-4bc0-f4bd-b931292ad30b"
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/\n",
      "Collecting mediapipe\n",
      "  Downloading mediapipe-0.9.1.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (33.0 MB)\n",
      "\u001b[2K     \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m33.0/33.0 MB\u001b[0m \u001b[31m32.7 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
      "\u001b[?25hRequirement already satisfied: flatbuffers>=2.0 in /usr/local/lib/python3.8/dist-packages (from mediapipe) (23.1.21)\n",
      "Requirement already satisfied: attrs>=19.1.0 in /usr/local/lib/python3.8/dist-packages (from mediapipe) (22.2.0)\n",
      "Requirement already satisfied: matplotlib in /usr/local/lib/python3.8/dist-packages (from mediapipe) (3.2.2)\n",
      "Requirement already satisfied: protobuf<4,>=3.11 in /usr/local/lib/python3.8/dist-packages (from mediapipe) (3.19.6)\n",
      "Requirement already satisfied: numpy in /usr/local/lib/python3.8/dist-packages (from mediapipe) (1.21.6)\n",
      "Requirement already satisfied: absl-py in /usr/local/lib/python3.8/dist-packages (from mediapipe) (1.4.0)\n",
      "Requirement already satisfied: opencv-contrib-python in /usr/local/lib/python3.8/dist-packages (from mediapipe) (4.6.0.66)\n",
      "Requirement already satisfied: kiwisolver>=1.0.1 in /usr/local/lib/python3.8/dist-packages (from matplotlib->mediapipe) (1.4.4)\n",
      "Requirement already satisfied: python-dateutil>=2.1 in /usr/local/lib/python3.8/dist-packages (from matplotlib->mediapipe) (2.8.2)\n",
      "Requirement already satisfied: cycler>=0.10 in /usr/local/lib/python3.8/dist-packages (from matplotlib->mediapipe) (0.11.0)\n",
      "Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /usr/local/lib/python3.8/dist-packages (from matplotlib->mediapipe) (3.0.9)\n",
      "Requirement already satisfied: six>=1.5 in /usr/local/lib/python3.8/dist-packages (from python-dateutil>=2.1->matplotlib->mediapipe) (1.15.0)\n",
      "Installing collected packages: mediapipe\n",
      "Successfully installed mediapipe-0.9.1.0\n"
     ]
    }
   ],
   "source": [
    "! pip install mediapipe"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "C2W8GpMY-QIR"
   },
   "source": [
    "Создадим класс для детекции лица и классификации жеста на фото. Логика работы метода run следующая:\n",
    "\n",
    "\n",
    "*   Детектируем с помощью mtcnn лицо и сохраняем рамку и маркеры лица.\n",
    "*   Детектируем руку на изображении с помощью mediapipe.\n",
    "\n",
    "\n",
    "*   Вырезаем часть изображения, где есть рука, по точкам детектора рук.\n",
    "*   Подаем часть изображения с рукой в обученный ранее классификатор жестов.\n",
    "\n",
    "\n",
    "*   Классифицируем жест и отрисовываем рамку на лице с маркерами лица, и подписываем рамку классифицированным жестом.\n",
    "\n",
    "\n",
    "\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "PLy6mzrQq5Da"
   },
   "outputs": [],
   "source": [
    "import cv2\n",
    "from PIL import Image\n",
    "from facenet_pytorch import MTCNN\n",
    "import mediapipe as mp\n",
    "\n",
    "    \n",
    "class FaceDetector(object):\n",
    "    \"\"\"\n",
    "    Face detector class\n",
    "    \"\"\"\n",
    "\n",
    "    def __init__(self, mtcnn, filename):\n",
    "        self.classes_name = ['01_palm',\n",
    "                              '02_l',\n",
    "                              '03_fist',\n",
    "                              '04_fist_moved',\n",
    "                              '05_thumb',\n",
    "                              '06_index',\n",
    "                              '07_ok',\n",
    "                              '08_palm_moved',\n",
    "                              '09_c',\n",
    "                              '10_down']\n",
    "        self.preprocess = tt.Compose([\n",
    "                         tt.Resize([128, 128]),\n",
    "                         tt.Grayscale(num_output_channels=3), # Картинки черно-белые\n",
    "                         tt.ToTensor(),                       # Приведение к тензору\n",
    "                         ]) \n",
    "        self.filename = filename\n",
    "        self.mtcnn = mtcnn\n",
    "        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'\n",
    "        self.model = torch.load('/content/drive/MyDrive/!!GeekBrains/31 Pythoch для разработки ИНС/!Пономарева/10 Распознавание лиц и эмоций/Курсовая/checkpoint_93.25')\n",
    "        self.model.eval()\n",
    "\n",
    "    def _draw(self, frame, boxes, probs, landmarks, gests):\n",
    "        \"\"\"\n",
    "        Draw landmarks and boxes for each face detected\n",
    "        \"\"\"\n",
    "        for box, prob, ld, gest in zip(boxes, probs, landmarks, gests):\n",
    "            # Draw rectangle on frame\n",
    "            cv2.rectangle(frame,\n",
    "                          (int(box[0]), int(box[1])),\n",
    "                          (int(box[2]), int(box[3])),\n",
    "                          (0, 0, 255),\n",
    "                          thickness=2)\n",
    "\n",
    "             # Show probability\n",
    "            cv2.putText(frame, \n",
    "                gest, (int(box[2]), int(box[3])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)\n",
    "            \n",
    "            ld = ld.astype(np.int64)\n",
    "\n",
    "             # Draw landmarks\n",
    "            cv2.circle(frame, tuple(ld[0]), 5, (0, 0, 255), -1)\n",
    "            cv2.circle(frame, tuple(ld[1]), 5, (0, 0, 255), -1)\n",
    "            cv2.circle(frame, tuple(ld[2]), 5, (0, 0, 255), -1)\n",
    "            cv2.circle(frame, tuple(ld[3]), 5, (0, 0, 255), -1)\n",
    "            cv2.circle(frame, tuple(ld[4]), 5, (0, 0, 255), -1)\n",
    "\n",
    "        return frame\n",
    "    \n",
    "\n",
    "    def digit_to_classname(self, digit):\n",
    "        for idx, itm in enumerate(self.classes_name):\n",
    "          if idx == digit:\n",
    "            return itm\n",
    "    \n",
    "    @staticmethod\n",
    "    def crop_hands(frame, hand_boxes):\n",
    "        hands = []\n",
    "        for i, hand_box in enumerate(hand_boxes):\n",
    "            pad = int((hand_box[2] - hand_box[0]) / 4)\n",
    "            h,w,c = frame.shape\n",
    "            pad1 = pad if hand_box[1]-pad >= 0 else hand_box[1]\n",
    "            pad2 = pad if hand_box[3]+pad <= h else h - hand_box[3]\n",
    "            pad3 = pad if hand_box[0]-pad >= 0 else hand_box[0]\n",
    "            pad4 = pad if hand_box[2]-pad <= w else w - hand_box[2]\n",
    "            hands.append(frame[int(hand_box[1]-pad1):int(hand_box[3]+pad2), \n",
    "                int(hand_box[0]-pad3):int(hand_box[2]+pad4), :])\n",
    "        return hands\n",
    "    \n",
    "    @staticmethod\n",
    "    def get_hands(img):\n",
    "      mpHands = mp.solutions.hands\n",
    "      hands = mpHands.Hands(False)\n",
    "      npDraw = mp.solutions.drawing_utils\n",
    "\n",
    "      results = hands.process(img)\n",
    "      if results.multi_hand_landmarks:\n",
    "          hand_boxes = []\n",
    "          for handLms in results.multi_hand_landmarks:\n",
    "              h,w,c = img.shape\n",
    "              x_max = 0\n",
    "              y_max = 0\n",
    "              x_min = w\n",
    "              y_min = h\n",
    "              for lm in handLms.landmark:\n",
    "                  x, y = int(lm.x * w), int(lm.y * h)\n",
    "                  if x > x_max:\n",
    "                      x_max = x\n",
    "                  if x < x_min:\n",
    "                      x_min = x\n",
    "                  if y > y_max:\n",
    "                      y_max = y\n",
    "                  if y < y_min:\n",
    "                      y_min = y\n",
    "              hand_box = [x_min, y_min, x_max, y_max]\n",
    "          hand_boxes.append(hand_box)\n",
    "      return hand_boxes\n",
    "      \n",
    "\n",
    "    def run(self, path_to_img):\n",
    "        \"\"\"\n",
    "            Run the FaceDetector and draw landmarks and boxes around detected faces\n",
    "        \"\"\"                \n",
    "        frame = cv2.imread(path_to_img)\n",
    "        \n",
    "        \n",
    "        # detect face box, probability and landmarks\n",
    "        boxes, probs, landmarks = self.mtcnn.detect(frame, landmarks=True)\n",
    "        print(boxes)\n",
    "\n",
    "        hand_boxes = self.get_hands(frame)\n",
    "        hands = self.crop_hands(frame, hand_boxes)\n",
    "\n",
    "        gests = []\n",
    "        for hand in hands:\n",
    "            hand = cv2.cvtColor(hand, cv2.COLOR_BGR2RGB)\n",
    "            hand = Image.fromarray(hand)\n",
    "            hand = self.preprocess(hand)\n",
    "            # hand = TF.adjust_brightness(hand, 1.5)\n",
    "            # hand = TF.adjust_contrast(hand, 1.5)\n",
    "            hand = hand.to(device).float()\n",
    "            gest = self.model(hand[None, ...])\n",
    "            print(gest)\n",
    "            gest = self.digit_to_classname(gest[0].argmax())\n",
    "            gests.append(gest)\n",
    "\n",
    "        # draw on frame\n",
    "        self._draw(frame, boxes, probs, landmarks, gests)\n",
    "    \n",
    "        # Show the frame\n",
    "        cv2.imwrite(self.filename, frame)\n",
    "        "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "33yxFhQSADEF"
   },
   "source": [
    "Функция для запуска камеры в GoogleCollab. Делает только один кадр из видеопотока. В конце этой функции Вызывается детектор лица с классификатором жестов рук."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "Sox-a65Njt89"
   },
   "outputs": [],
   "source": [
    "from IPython.display import display, Javascript\n",
    "from google.colab.output import eval_js\n",
    "from base64 import b64decode\n",
    "\n",
    "def take_photo(filename='photo.jpg', quality=0.8):\n",
    "  js = Javascript('''\n",
    "    async function takePhoto(quality) {\n",
    "      const div = document.createElement('div');\n",
    "      const capture = document.createElement('button');\n",
    "      capture.textContent = 'Capture';\n",
    "      div.appendChild(capture);\n",
    "\n",
    "      const video = document.createElement('video');\n",
    "      video.style.display = 'block';\n",
    "      const stream = await navigator.mediaDevices.getUserMedia({video: true});\n",
    "\n",
    "      document.body.appendChild(div);\n",
    "      div.appendChild(video);\n",
    "      video.srcObject = stream;\n",
    "      await video.play();\n",
    "\n",
    "      // Resize the output to fit the video element.\n",
    "      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);\n",
    "\n",
    "      // Wait for Capture to be clicked.\n",
    "      await new Promise((resolve) => capture.onclick = resolve);\n",
    "\n",
    "      const canvas = document.createElement('canvas');\n",
    "      canvas.width = video.videoWidth;\n",
    "      canvas.height = video.videoHeight;\n",
    "      canvas.getContext('2d').drawImage(video, 0, 0);\n",
    "      stream.getVideoTracks()[0].stop();\n",
    "      div.remove();\n",
    "      return canvas.toDataURL('image/jpeg', quality);\n",
    "    }\n",
    "    ''')\n",
    "  display(js)\n",
    "  data = eval_js('takePhoto({})'.format(quality))\n",
    "  binary = b64decode(data.split(',')[1])\n",
    "  with open(filename, 'wb') as f:\n",
    "    f.write(binary)\n",
    "  \n",
    "  mtcnn = MTCNN()\n",
    "\n",
    "  fcd = FaceDetector(mtcnn, filename)\n",
    "\n",
    "  fcd.run(filename)\n",
    "\n",
    "  return filename"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "uHaw3U-dAxfd"
   },
   "source": [
    "Функция обработки фотографии из видеопотока, и вывода обработанной фотографии:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "rHm7dc9Sjt9B"
   },
   "outputs": [],
   "source": [
    "from IPython.display import Image as Img\n",
    "def detect_face_and_cls_gest():\n",
    "  try:\n",
    "    filename = take_photo()\n",
    "    print('Saved to {}'.format(filename))\n",
    "    \n",
    "    # Show the image which was just taken.\n",
    "    display(Img(filename))\n",
    "  except Exception as err:\n",
    "    # Errors will be thrown if the user does not have a webcam or if they do not\n",
    "    # grant the page permission to access it.\n",
    "    print(str(err))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "cSc11eLoBPHw"
   },
   "source": [
    "Примеры работы функции:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/",
     "height": 589
    },
    "id": "hMe3Cm2og2YH",
    "outputId": "cec6e8fb-322a-4a60-a046-89a6236755d3"
   },
   "outputs": [],
   "source": [
    "detect_face_and_cls_gest()"
   ]
  }
 ],
 "metadata": {
  "accelerator": "GPU",
  "colab": {
   "provenance": []
  },
  "gpuClass": "standard",
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.16"
  },
  "widgets": {
   "application/vnd.jupyter.widget-state+json": {
    "094c7a001c1049c786160cb67562712c": {
     "model_module": "@jupyter-widgets/controls",
     "model_module_version": "1.5.0",
     "model_name": "HTMLModel",
     "state": {
      "_dom_classes": [],
      "_model_module": "@jupyter-widgets/controls",
      "_model_module_version": "1.5.0",
      "_model_name": "HTMLModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/controls",
      "_view_module_version": "1.5.0",
      "_view_name": "HTMLView",
      "description": "",
      "description_tooltip": null,
      "layout": "IPY_MODEL_7267bdf841bf41acb9cd60eefe3e9250",
      "placeholder": "​",
      "style": "IPY_MODEL_cd83185ca5a64d1c86c8e296a9ba0275",
      "value": " 97.8M/97.8M [00:01&lt;00:00, 81.8MB/s]"
     }
    },
    "145a4210965b4cb182df9e65c6a83838": {
     "model_module": "@jupyter-widgets/base",
     "model_module_version": "1.2.0",
     "model_name": "LayoutModel",
     "state": {
      "_model_module": "@jupyter-widgets/base",
      "_model_module_version": "1.2.0",
      "_model_name": "LayoutModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/base",
      "_view_module_version": "1.2.0",
      "_view_name": "LayoutView",
      "align_content": null,
      "align_items": null,
      "align_self": null,
      "border": null,
      "bottom": null,
      "display": null,
      "flex": null,
      "flex_flow": null,
      "grid_area": null,
      "grid_auto_columns": null,
      "grid_auto_flow": null,
      "grid_auto_rows": null,
      "grid_column": null,
      "grid_gap": null,
      "grid_row": null,
      "grid_template_areas": null,
      "grid_template_columns": null,
      "grid_template_rows": null,
      "height": null,
      "justify_content": null,
      "justify_items": null,
      "left": null,
      "margin": null,
      "max_height": null,
      "max_width": null,
      "min_height": null,
      "min_width": null,
      "object_fit": null,
      "object_position": null,
      "order": null,
      "overflow": null,
      "overflow_x": null,
      "overflow_y": null,
      "padding": null,
      "right": null,
      "top": null,
      "visibility": null,
      "width": null
     }
    },
    "658e31dcb92542eb8ea75acb82c8acec": {
     "model_module": "@jupyter-widgets/controls",
     "model_module_version": "1.5.0",
     "model_name": "FloatProgressModel",
     "state": {
      "_dom_classes": [],
      "_model_module": "@jupyter-widgets/controls",
      "_model_module_version": "1.5.0",
      "_model_name": "FloatProgressModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/controls",
      "_view_module_version": "1.5.0",
      "_view_name": "ProgressView",
      "bar_style": "success",
      "description": "",
      "description_tooltip": null,
      "layout": "IPY_MODEL_ab85f01ff9f646d39c86bd389f4cc7cf",
      "max": 102530333,
      "min": 0,
      "orientation": "horizontal",
      "style": "IPY_MODEL_831660235cd4450e9e92f09cfdb0d9bc",
      "value": 102530333
     }
    },
    "718db746d0e6410d9ad5223df0ca300e": {
     "model_module": "@jupyter-widgets/base",
     "model_module_version": "1.2.0",
     "model_name": "LayoutModel",
     "state": {
      "_model_module": "@jupyter-widgets/base",
      "_model_module_version": "1.2.0",
      "_model_name": "LayoutModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/base",
      "_view_module_version": "1.2.0",
      "_view_name": "LayoutView",
      "align_content": null,
      "align_items": null,
      "align_self": null,
      "border": null,
      "bottom": null,
      "display": null,
      "flex": null,
      "flex_flow": null,
      "grid_area": null,
      "grid_auto_columns": null,
      "grid_auto_flow": null,
      "grid_auto_rows": null,
      "grid_column": null,
      "grid_gap": null,
      "grid_row": null,
      "grid_template_areas": null,
      "grid_template_columns": null,
      "grid_template_rows": null,
      "height": null,
      "justify_content": null,
      "justify_items": null,
      "left": null,
      "margin": null,
      "max_height": null,
      "max_width": null,
      "min_height": null,
      "min_width": null,
      "object_fit": null,
      "object_position": null,
      "order": null,
      "overflow": null,
      "overflow_x": null,
      "overflow_y": null,
      "padding": null,
      "right": null,
      "top": null,
      "visibility": null,
      "width": null
     }
    },
    "7267bdf841bf41acb9cd60eefe3e9250": {
     "model_module": "@jupyter-widgets/base",
     "model_module_version": "1.2.0",
     "model_name": "LayoutModel",
     "state": {
      "_model_module": "@jupyter-widgets/base",
      "_model_module_version": "1.2.0",
      "_model_name": "LayoutModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/base",
      "_view_module_version": "1.2.0",
      "_view_name": "LayoutView",
      "align_content": null,
      "align_items": null,
      "align_self": null,
      "border": null,
      "bottom": null,
      "display": null,
      "flex": null,
      "flex_flow": null,
      "grid_area": null,
      "grid_auto_columns": null,
      "grid_auto_flow": null,
      "grid_auto_rows": null,
      "grid_column": null,
      "grid_gap": null,
      "grid_row": null,
      "grid_template_areas": null,
      "grid_template_columns": null,
      "grid_template_rows": null,
      "height": null,
      "justify_content": null,
      "justify_items": null,
      "left": null,
      "margin": null,
      "max_height": null,
      "max_width": null,
      "min_height": null,
      "min_width": null,
      "object_fit": null,
      "object_position": null,
      "order": null,
      "overflow": null,
      "overflow_x": null,
      "overflow_y": null,
      "padding": null,
      "right": null,
      "top": null,
      "visibility": null,
      "width": null
     }
    },
    "79ad86576b824112b04c7b993b96fd7a": {
     "model_module": "@jupyter-widgets/controls",
     "model_module_version": "1.5.0",
     "model_name": "HBoxModel",
     "state": {
      "_dom_classes": [],
      "_model_module": "@jupyter-widgets/controls",
      "_model_module_version": "1.5.0",
      "_model_name": "HBoxModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/controls",
      "_view_module_version": "1.5.0",
      "_view_name": "HBoxView",
      "box_style": "",
      "children": [
       "IPY_MODEL_e34eb8bc9d444d14b9a31cd8537ee725",
       "IPY_MODEL_658e31dcb92542eb8ea75acb82c8acec",
       "IPY_MODEL_094c7a001c1049c786160cb67562712c"
      ],
      "layout": "IPY_MODEL_718db746d0e6410d9ad5223df0ca300e"
     }
    },
    "831660235cd4450e9e92f09cfdb0d9bc": {
     "model_module": "@jupyter-widgets/controls",
     "model_module_version": "1.5.0",
     "model_name": "ProgressStyleModel",
     "state": {
      "_model_module": "@jupyter-widgets/controls",
      "_model_module_version": "1.5.0",
      "_model_name": "ProgressStyleModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/base",
      "_view_module_version": "1.2.0",
      "_view_name": "StyleView",
      "bar_color": null,
      "description_width": ""
     }
    },
    "ab85f01ff9f646d39c86bd389f4cc7cf": {
     "model_module": "@jupyter-widgets/base",
     "model_module_version": "1.2.0",
     "model_name": "LayoutModel",
     "state": {
      "_model_module": "@jupyter-widgets/base",
      "_model_module_version": "1.2.0",
      "_model_name": "LayoutModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/base",
      "_view_module_version": "1.2.0",
      "_view_name": "LayoutView",
      "align_content": null,
      "align_items": null,
      "align_self": null,
      "border": null,
      "bottom": null,
      "display": null,
      "flex": null,
      "flex_flow": null,
      "grid_area": null,
      "grid_auto_columns": null,
      "grid_auto_flow": null,
      "grid_auto_rows": null,
      "grid_column": null,
      "grid_gap": null,
      "grid_row": null,
      "grid_template_areas": null,
      "grid_template_columns": null,
      "grid_template_rows": null,
      "height": null,
      "justify_content": null,
      "justify_items": null,
      "left": null,
      "margin": null,
      "max_height": null,
      "max_width": null,
      "min_height": null,
      "min_width": null,
      "object_fit": null,
      "object_position": null,
      "order": null,
      "overflow": null,
      "overflow_x": null,
      "overflow_y": null,
      "padding": null,
      "right": null,
      "top": null,
      "visibility": null,
      "width": null
     }
    },
    "c99e49879b194cb4b62b8fb589cd942b": {
     "model_module": "@jupyter-widgets/controls",
     "model_module_version": "1.5.0",
     "model_name": "DescriptionStyleModel",
     "state": {
      "_model_module": "@jupyter-widgets/controls",
      "_model_module_version": "1.5.0",
      "_model_name": "DescriptionStyleModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/base",
      "_view_module_version": "1.2.0",
      "_view_name": "StyleView",
      "description_width": ""
     }
    },
    "cd83185ca5a64d1c86c8e296a9ba0275": {
     "model_module": "@jupyter-widgets/controls",
     "model_module_version": "1.5.0",
     "model_name": "DescriptionStyleModel",
     "state": {
      "_model_module": "@jupyter-widgets/controls",
      "_model_module_version": "1.5.0",
      "_model_name": "DescriptionStyleModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/base",
      "_view_module_version": "1.2.0",
      "_view_name": "StyleView",
      "description_width": ""
     }
    },
    "e34eb8bc9d444d14b9a31cd8537ee725": {
     "model_module": "@jupyter-widgets/controls",
     "model_module_version": "1.5.0",
     "model_name": "HTMLModel",
     "state": {
      "_dom_classes": [],
      "_model_module": "@jupyter-widgets/controls",
      "_model_module_version": "1.5.0",
      "_model_name": "HTMLModel",
      "_view_count": null,
      "_view_module": "@jupyter-widgets/controls",
      "_view_module_version": "1.5.0",
      "_view_name": "HTMLView",
      "description": "",
      "description_tooltip": null,
      "layout": "IPY_MODEL_145a4210965b4cb182df9e65c6a83838",
      "placeholder": "​",
      "style": "IPY_MODEL_c99e49879b194cb4b62b8fb589cd942b",
      "value": "100%"
     }
    }
   }
  }
 },
 "nbformat": 4,
 "nbformat_minor": 1
} 

{'cells': [{'cell_type': 'markdown',
   'metadata': {'id': '14J-ylhuxukV'},
   'source': ['# Урок 10. Распознавание лиц и эмоций.\n',
    '## **Задание по итогам курса:**\n',
    '\n',
    '- (упрощенное/для тех, у кого нет вебкамеры)\n',
    '    1. Нужно написать приложение, которое будет получать на вход *изображение*.\n',
    '    2. В процессе определять, что перед камерой находится человек, задетектировав его лицо на кадре.\n',
    '    3. На изображении человек показывает жесты руками, а алгоритм должен считать их и классифицировать.\n',
    '\n',
    '- (более сложное)\n',
    '    1. Нужно написать приложение, которое будет считывать и выводить кадры с *веб-камеры*.\n',
    '    2. В процессе считывания определять что перед камерой находится человек, задетектировав его лицо на кадре.\n',
    '    3. Человек показывает жесты руками, а алгоритм должен считать их и классифицировать. \n',
    '____________________\n',
    '- Для распознавания жестов, вам надо будет скачать датасет